# Si446x Device Direct Access Ping

In [ ]:
%autosave 0
import sys
sys.path.append("../") # go to parent dir
%run si446x_Device_Layer.ipynb
#test

In [ ]:
import datetime
print('Test Start Time: {}'.format(datetime.datetime.now()))
print('Si446x Radio Device Driver Version: {}'.format(si446x_device_version()))

##  Start up Radio

In [ ]:
radio = si446x_device_start_radio()

In [ ]:
si446x_device_show_config(radio.dump_radio())

## Check for Command Error

In [ ]:
status = radio.get_chip_status()
if (status.chip_pend.CMD_ERROR):
    print(status)

##  Configure Radio

In [ ]:
config = si446x_device_config_radio(radio)

si446x_device_show_config(radio.dump_radio())
total = 0
print('\n=== const config strings:')
for s in config:
    print((hexlify(s)))
    total += len(s) - 4
print('\n total: {}'.format(total))

## Get Chip Status

In [ ]:
print(radio.get_chip_status())

## Interactive Group Properties

In [ ]:
interact(si446x_device_group_fetch_and_decode, group=radio_config_group_ids.encoding)

## Interactive  Command Status Responses

In [ ]:
interact(si446x_device_command_fetch_and_decode, cmd=radio_status_cmd_ids.encoding)

## Transmit Packet Stream

In [ ]:
#
# typedef struct test_msg {
#    uint8_t       len;
#    uint8_t       seq;
#    uint8_t       pa;
#    uint8_t       rssi;
#    uint16_t      addr;
#    test_mode_t   mode;
#    uint8_t       data[0];
#  } PACKED test_msg_t;


In [ ]:
from __future__ import print_function
from time import sleep
from datetime import datetime


num         = 1000000
MAX_WAIT    = 1

MAX_RECV    = 256
msg         = bytearray(MAX_RECV)
for x in range(MAX_RECV): msg[x]=x    

mj_r        = range(num)
th_r        = range(40,41,5)              # fifo threshold setting (min,max,opt): 1,64,40
sz_r        = range(10,259,5)             # msg size (min,max): 4, 255
pw_r        = [1]
#pw_r        = [1 << i for i in range(2)] # power amp setting (min, max,opt): 1,127,10
delay       = .01

prp         = bytearray(1)

xcnt        = 0
rcnt        = 0
pstr        = ''

rssi        = 0

for i in range(len(msg)): msg[i] = i
radio.trace._enable()

prp[0] = 0x22
radio.set_property('MODEM', 0x4c, prp) 

start = datetime.now()
print(start)

for rep in range(num):
    for thresh in th_r:
        prp[0]=thresh
        radio.set_property('PKT', 0x0b, prp)  # set tx fifo threshhold
        for pwr in pw_r:
            if (pwr > 127): pwr = 127
            for size in sz_r:
                if (size > 255): size = 255
                msg[0] = size
                msg[1] = (xcnt % 256)
                msg[2] = pwr
                msg[3] = rssi
                l = si446x_device_send_msg(radio, msg[:size], pwr)
                rmsg, rssi, status = si446x_device_receive_msg(radio, size, MAX_WAIT)
                xcnt += 1
                if (rmsg):
                    rcnt += 1
                    last_len = len(pstr) if (pstr) else 0
                    pstr = '{}, xc/rc:{}/{}, l:{}, t:{}, lp/rr:{}/{}, rp/lr:{}/{}, m:{}..{}'.format(
                                datetime.now(), xcnt, rcnt, size, thresh,
                                pwr, rmsg[3], rmsg[2], rssi, hexlify(rmsg[:8]), hexlify(rmsg[-8:]))
                else:
                    pstr = '{}, xc/rc:{}/{}, l:{}, t:{}, lp/rr:{}/{}, rp/lr:{}/{}, m:{}'.format(
                                datetime.now(), xcnt, rcnt, size, thresh,
                                pwr, -1, -1, rssi, hexlify(msg[:4]))
                print(pstr, end='')
                if (status[-1] == 'e'):
                    print(', {}'.format(''.join(map(str, status))))
                else:
                    x = last_len - len(pstr) if (last_len > len(pstr)) else 1
                    print(' ' * x,end='\r')
                sleep(delay)
print('\ndone')

In [ ]:
##OLD##

from __future__ import print_function
from time import sleep
from datetime import datetime

num         = 10000
delay       = .3
pkt         = bytearray(256)

step        = 1
prp         = bytearray(1)
mj_r        = range(num)
th_r        = range(40,41,5)   # fifo threshold setting (min,max): 1, 64
sz_r        = range(2,260,5)  # msg size (min,max): 4, 255
#pw_r        = range(1,15)  # power amp setting (min, max): 1, 127
pw_r        = [1 << i for i in range(8)]
count       = 0
radio.trace._enable()

start = datetime.now()
print(start)

for major in mj_r:
    for thresh in th_r:
        prp[0]=thresh
        radio.set_property('PKT', 0x0b, prp)  # set tx fifo threshhold
        for pw in pw_r:
            if (pw > 127): pw = 127
            for size in sz_r:
                if (size > 255): size = 255
                pkt[1] = (size % 256)
                l = si446x_device_send_msg(radio, pkt[:size], pw)
                sleep(delay)
                count += 1
                print('\r{}, c:{}, l:{}, p:{}, t:{} s:{}'.format(
                    datetime.now(), count, size, pw, thresh, ''.join(map(str, l))),end='')
                if (l[-1] == 'e'): print()
                else: print('                    ',end='')
print('\ndone')

## Check Interrupt Status

In [ ]:
s_name =  'int_status_rsp_s'
p_s = eval(s_name)
p_g = radio.get_interrupts()
p_d = p_s.build(p_g)
print('{}: {}'.format(s_name, insert_space(p_d)))
print(radio_display_structs[p_s](p_s, p_d))

In [ ]:
s_name =  'get_chip_status_rsp_s'
p_s = eval(s_name)
p_g = radio.get_chip_status()
p_d = p_s.build(p_g)
print('{}: {}'.format(s_name, insert_space(p_d)))
print(radio_display_structs[p_s](p_s, p_d))

In [ ]:
#t,r = si446xradio_test()

In [ ]:
from datetime import datetime 

now=datetime.now()
print(now)

In [ ]:
MAX_CHUNK=64

In [ ]:
radio.clear_interrupts()

In [ ]:
rx, tx = radio.fifo_info(rx_flush=True, tx_flush=True)
print(rx,tx)

In [ ]:
radio.set_property('PKT', 0x0b, b'\x3c')  # set tx fifo threshhold

In [ ]:
rx, tx = radio.fifo_info()
print(rx,tx)

In [ ]:
status = radio.get_interrupts()
print(status.ph_pend.TX_FIFO_ALMOST_EMPTY)

In [ ]:
s_name =  'int_status_rsp_s'
p_s = eval(s_name)
p_g = radio.get_interrupts()
p_d = p_s.build(p_g)
print('{}: {}'.format(s_name, insert_space(p_d)))
print(radio_display_structs[p_s](p_s, p_d))

In [ ]:
buf = bytearray('\x22' * 64)
chunk = 8
cur   = 8
radio.write_tx_fifo(buf[:chunk])
rx, tx = radio.fifo_info()
print(rx,tx)

In [ ]:
radio.start_tx(len(buf))
while (True):
    status = radio.get_interrupts()
    if (status.ph_pend.TX_FIFO_ALMOST_EMPTY):
        rx, tx = radio.fifo_info()
        radio.write_tx_fifo(buf[cur:cur+chunk])
        cur += chunk
        print('.',end='')
    if (status.ph_pend.PACKET_SENT):
        print('done')
        break
    radio.clear_interrupts()

In [ ]:
ss[0] = 10
ss[1] = 20

In [ ]:
cur

In [ ]:
radio.set_property('PKT', 0x0b, ss) # tx/rx threshold

In [ ]:
radio.set_property('PKT', 0x0b, b'\x48')  # set tx fifo threshhold

In [ ]:
radio.set_property('PKT', 0x0c, b'\x14')  # set rx fifo threshold

In [ ]:
status = int_status(clr_all_flags, False)
print(status.ph_pend.TX_FIFO_ALMOST_EMPTY)